In [1]:
DEVICE = 'cuda:0'
PATH = '../models/chatglm2/'

In [2]:
import numpy as np
import pandas as pd 
import torch
from torch import nn 
from torch.utils.data import Dataset,DataLoader 

In [3]:

from argparse import Namespace
cfg = Namespace()

#dataset
cfg.prompt_column = 'prompt'     # 数据集中包含提示文本的列的名称
cfg.response_column = 'response' # 数据集中包含响应文本的列的名称
cfg.history_column = None        # 
cfg.source_prefix = ''           # 添加到每个prompt开头的前缀引导语

cfg.max_source_length = 192 # 提示文本长度
cfg.max_target_length = 256 # 响应文本长度

# model
cfg.model_name_or_path = PATH # 远程'THUDM/chatglm-6b' 
cfg.quantization_bit = None   # 量化 


#train
cfg.epochs = 100 
# cfg.lr = 5e-3
cfg.lr = 1e-3
cfg.batch_size = 1
cfg.gradient_accumulation_steps = 16 #梯度累积

- 读取模型

In [4]:
import transformers
from transformers import  AutoModel,AutoTokenizer,AutoConfig,DataCollatorForSeq2Seq


config = AutoConfig.from_pretrained(cfg.model_name_or_path, trust_remote_code=True)

tokenizer = AutoTokenizer.from_pretrained(
    cfg.model_name_or_path, trust_remote_code=True)

model = AutoModel.from_pretrained(cfg.model_name_or_path,config=config,
                                  trust_remote_code=True).cuda(DEVICE)

/opt/conda/envs/chatglm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 7/7 [00:10<00:00,  1.45s/it]


- 构建chat函数
- - 输入模型、文本、历史记录、温度
- - 返回回答和历史

In [5]:
def chat(model,text,his,temperature = 0.01):
    ans,his = model.chat(tokenizer, text, history=his, temperature=temperature)
    return ans,his

- 做zero-shot的实验
- - 通过不同的温度查看可能的结果

In [6]:
ans,his_1 = chat(model,'你知道林黛玉倒拔垂杨柳吗？请详细说说这故事。',his=[],temperature = 0.95)
print(ans)

我知道这个故事，它讲述了林黛玉和垂杨柳的故事。

据传，林黛玉是中国古典名著《红楼梦》中的主人公之一。她是一个文艺青年，有着出色的诗歌才华，但最终因为悲痛过度而去世。

垂杨柳是古典名著《水浒传》中的一个角色，他是一个反抗压迫的英雄。据传，他曾经倒拔垂杨柳，以表达他对社会的不满和反抗。

虽然这两个故事的主人公不同，但它们都包含了一个英雄倒拔的形象，这可能是因为在古典文学中，英雄们通常会以倒拔垂杨柳等形象来表达他们的勇气和决心。


In [7]:
ans,his_2 = chat(model,'你觉得你刚刚说的对吗？你知道林黛玉倒拔垂杨柳吗？',his=his_1,temperature = 0.95)
print(ans)

我刚刚说错了，我并没有提到林黛玉和垂杨柳的故事。

据我所知，林黛玉是《红楼梦》中的一个角色，而垂杨柳是《水浒传》中的一个角色。它们两个人之间并没有任何联系。

《红楼梦》是一部描写清朝贵族生活的长篇小说，而《水浒传》是一部描写宋朝农民起义的长篇小说。虽然它们都是古典名著，但它们的故事和人物之间并没有任何联系。


In [8]:
ans,his_3 = chat(model,'真的吗？真的有这故事吗？',his=his_2,temperature = 0.95)
print(ans)

据我所知，并没有确凿的证据证明林黛玉和垂杨柳之间有故事。

在《红楼梦》中，林黛玉是贾宝玉的表妹，她是一个文艺青年，有着出色的诗歌才华。她最终因为悲痛过度而去世，但她的故事并没有提到垂杨柳。

在《水浒传》中，垂杨柳是宋江的义弟，他是一个反抗压迫的英雄。据说，他曾经倒拔垂杨柳，但这个故事和林黛玉并没有任何联系。

虽然《红楼梦》和《水浒传》都是古典名著，但它们的故事和人物之间并没有任何联系。


In [9]:
ans,his = chat(model,'根据我们的对话，你知道林黛玉倒拔垂杨柳吗？请详细说说这故事。',his=his_3,temperature = 0.95)
print(ans)

我很抱歉，我之前说错了。据我所知，林黛玉和垂杨柳之间并没有任何联系。

在《红楼梦》中，林黛玉是贾宝玉的表妹，她是一个文艺青年，有着出色的诗歌才华。她最终因为悲痛过度而去世，但她的故事并没有提到垂杨柳。

在《水浒传》中，垂杨柳是宋江的义弟，他是一个反抗压迫的英雄。据说，他曾经倒拔垂杨柳，但这个故事和林黛玉并没有任何联系。

虽然《红楼梦》和《水浒传》都是古典名著，但它们的故事和人物之间并没有任何联系。
